In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm.notebook import tqdm

In [127]:
course = pd.read_csv("data/course.csv") 
course_element = pd.read_csv("data/course_element.csv") 
course_module = pd.read_csv("data/course_module.csv") 
sol_log = pd.read_csv("data/solution_log.csv") 
u_course_progress = pd.read_csv("data/user_course_progress.csv") 
u_element_progress = pd.read_csv("data/user_element_progress.csv") 
u_module_progress = pd.read_csv("data/user_module_progress.csv")

C:\Users\Sirius\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Sirius\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [128]:
## Посчитаем, сколько в курсе можно заработать максимум баллов
course_to_max_points = dict(course_module.groupby("course_id").progress_max.sum())


## Подсчитаем, закрыл ли пользователь курс на сертификат первого уровня

d = dict(course_module[~course_module.is_advanced & course_module.level == 1].groupby(by="course_id").id.unique().apply(len))
u_course_progress["course_done"] = u_course_progress.course_id.map(d) <= u_course_progress.modules_achieved


## Подсчитаем по пользователю, сколько баллов он заработал

u_course_progress = u_course_progress.merge(u_element_progress.groupby(['user_id', 'course_id']).\
                                            progress_current.sum(), on=['user_id', 'course_id'])

## Найдем долю баллов которую получил пользователь от максимального числа баллов

u_course_progress["progress_current_ratio"] = u_course_progress.progress_current / u_course_progress.course_id.map(course_to_max_points)

u_course_progress.to_csv("user_course_progress_done.csv", index=None)

In [130]:
u_course_progress.groupby("user_id")

,id,user_id,course_id,modules_achieved,advanced_modules_done,time_created,time_updated,modules_closed,modules_done,course_done,progress_current,progress_current_ratio
0,2020,100120050111530528,3,0,0,NaN,2019-06-05 16:45:32.140810,0,0,False,0.0,0.000000
1,43519,100120050110043028,3,0,0,2019-05-06 19:22:12.861890,2019-06-05 16:45:32.140810,0,0,False,5.0,0.054348
2,26692,100120030111070018,3,0,0,2019-04-15 11:30:32.944271,2019-06-05 16:45:32.140810,0,0,False,0.0,0.000000
3,787,100120060114846069,3,0,0,NaN,2019-06-05 16:45:32.140810,0,0,False,0.0,0.000000
4,2681,100120060110982009,3,8,2,2019-03-22 15:42:21.255839,2019-06-14 11:48:47.039643,1,2,False,74.0,0.804348
...,...,...,...,...,...,...,...,...,...,...,...,...
26179,202673,100119790111022302,237,0,0,2020-09-16 02:25:52.295517,NaN,0,0,False,0.0,0.000000
26180,202230,100120060110010662,237,0,0,2020-09-15 04:10:34.778827,NaN,0,0,False,0.0,0.000000
26181,200338,100119750110038578,237,0,0,2020-09-11 12:29:43.569905,NaN,0,0,False,0.0,0.000000
26182,199512,100120040111161458,237,0,0,2020-09-10 09:04:36.023425,NaN,0,0,False,0.0,0.000000


In [129]:
u_course_progress = pd.read_csv("user_course_progress_done.csv") 

In [35]:
u_course_progress["achived"] = u_course_progress.course_id.map(d) == u_course_progress.modules_achieved

False    23758
True      2426
Name: achived, dtype: int64

In [16]:
u_module_progress.head()

,id,user_id,course_id,course_module_id,progress_current,progress_failed,steps_done,is_achieved,is_closed,course_progress_id,time_created,time_updated,time_closed,time_achieved,time_unlocked,tasks_done,time_done,is_done,achieve_reason
0,503562,100120050110012222,3,52,0.0,0.0,0,False,False,30298,2019-05-01 16:39:05.615147,NaN,NaN,NaN,NaN,0,NaN,False,NaN
1,46805,100120050111530528,3,59,0.0,0.0,0,False,False,2020,NaN,NaN,NaN,NaN,NaN,0,NaN,False,NaN
2,46803,100120050111530528,3,58,0.0,0.0,0,False,False,2020,NaN,NaN,NaN,NaN,NaN,0,NaN,False,NaN
3,46798,100120050111530528,3,57,0.0,0.0,0,False,False,2020,NaN,NaN,NaN,NaN,NaN,0,NaN,False,NaN
4,482893,100120050110594123,3,52,9.0,0.0,20,True,True,29023,2019-04-28 05:34:50.545702,2019-05-01 13:21:08.149917,2019-05-01 13:21:08.149917,2019-05-01 13:13:52.193671,2019-04-30 01:50:19.883097,9,2019-05-01 13:20:23.467270,True,NaN
